# Practice binarizing labels

In [1]:
import sys

sys.path.append('..')

In [2]:
import io
from functools import partial
from pathlib import Path
from tempfile import NamedTemporaryFile

import cv2 as cv
import pytesseract
from ipywidgets import interact
import numpy as np
from skimage import filters, morphology, restoration, util
from PIL import Image as PImage, ImageDraw, ImageOps
from wand.image import Image
from wand.drawing import Drawing

In [3]:
DATA_DIR = Path('..') / 'data'
LABELS_DIR = DATA_DIR / 'labels' / 'typewritten'
LABELS_DIR = DATA_DIR / 'labels' / 'handwritten'

### Raw image

In [4]:
IMAGES = sorted(LABELS_DIR.glob('*.jpg'))
print(IMAGES[46])

../data/labels/handwritten/10667627.jpg


In [5]:
def raw_image(idx):
    image = PImage.open(IMAGES[idx])
    display(image)


# raw_image(46)
interact(raw_image, idx=(0, len(IMAGES) - 1))

interactive(children=(IntSlider(value=543, description='idx', max=1087), Output()), _dom_classes=('widget-inte…

<function __main__.raw_image(idx)>

### Try scikit-image

In [6]:
BLOCK_SIZE = 11
OFFSET = 11


def scikit_image(idx):
    path = IMAGES[idx]

    image = PImage.open(path).convert('L')
    data = np.array(image)

    # threshold = filters.threshold_local(data, block_size=BLOCK_SIZE, offset=OFFSET)
    threshold = filters.threshold_sauvola(data, window_size=OFFSET, k=0.032)
    binary = data > threshold

    text = pytesseract.image_to_string(binary)

    print(path, '\n')
    print(text)

    display(PImage.fromarray(binary))


# scikit_image(46)
interact(scikit_image, idx=(0, len(IMAGES) - 1))

interactive(children=(IntSlider(value=543, description='idx', max=1087), Output()), _dom_classes=('widget-inte…

<function __main__.scikit_image(idx)>

### Try opencv

In [7]:
def opencv_image(idx):
    image = cv.imread(str(IMAGES[idx]))
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    # thresh = cv.adaptiveThreshold(
    #     image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 15, 16)
    thresh = cv.adaptiveThreshold(
        image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 15, 12)
    text = pytesseract.image_to_string(thresh)
    print(text)
    display(PImage.fromarray(thresh))


# opencv_image(46)
interact(opencv_image, idx=(0, len(IMAGES) - 1))

interactive(children=(IntSlider(value=543, description='idx', max=1087), Output()), _dom_classes=('widget-inte…

<function __main__.opencv_image(idx)>

### Try wand

In [8]:
radius = 11


def wand_image(idx):
    path = IMAGES[idx]
    with Image(filename=path) as image:
        image.transform_colorspace('gray')
        image.deskew(0.4)
        image.adaptive_threshold(
            width=radius, height=radius, offset=-0.04 * image.quantum_range)
        image.despeckle()
        image.sharpen()
        blob = image.make_blob('jpeg')
        blob = PImage.open(io.BytesIO(blob))
        text = pytesseract.image_to_string(blob)
        print(text)
        display(image)


# wand_image(46)
interact(wand_image, idx=(0, len(IMAGES) - 1))

interactive(children=(IntSlider(value=543, description='idx', max=1087), Output()), _dom_classes=('widget-inte…

<function __main__.wand_image(idx)>

In [9]:
%%javascript
$('.output_text').css('font-size', '16pt')

<IPython.core.display.Javascript object>